In [1]:
import pandas as pd
import numpy as np
import pickle
import scipy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, LogisticRegressionCV, SGDClassifier, RidgeClassifier
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, mean_absolute_error, f1_score
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import svm, linear_model
from sklearn import tree, metrics
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import gzip

In [2]:
# read in csv
df = pd.read_csv('training_25_features.csv')
df

,I,II,III,aVF,aVR,aVL,V1,V2,V3,V4,...,weight,nurse,site,device,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,-0.005,0.135,0.140,-0.065,-0.073,0.137,-0.125,-0.090,-0.110,-0.210,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
1,-0.005,0.135,0.140,-0.065,-0.073,0.137,-0.125,-0.090,-0.110,-0.211,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
2,-0.005,0.131,0.136,-0.063,-0.070,0.133,-0.125,-0.082,-0.102,-0.190,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
3,-0.005,0.130,0.135,-0.063,-0.070,0.132,-0.122,-0.077,-0.094,-0.172,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
4,-0.005,0.128,0.133,-0.062,-0.069,0.130,-0.119,-0.071,-0.084,-0.157,...,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4319171,0.010,0.170,0.160,-0.090,-0.075,0.165,0.155,0.365,0.230,0.030,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4319172,0.014,0.174,0.160,-0.094,-0.073,0.167,0.155,0.368,0.245,0.029,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4319173,0.016,0.176,0.160,-0.096,-0.073,0.167,0.155,0.383,0.261,0.040,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8
4319174,0.014,0.174,0.160,-0.094,-0.073,0.167,0.155,0.406,0.282,0.059,...,69.841845,1.0,2.0,1,3.0,1.0,0,1,0.0,8


In [3]:
# convert all the columns to float64
for i in range(26):
    df[df.columns[i]] = df[df.columns[i]].astype('float64')
    
# get info for columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4319176 entries, 0 to 4319175
Data columns (total 26 columns):
 #   Column              Dtype  
---  ------              -----  
 0   I                   float64
 1   II                  float64
 2   III                 float64
 3   aVF                 float64
 4   aVR                 float64
 5   aVL                 float64
 6   V1                  float64
 7   V2                  float64
 8   V3                  float64
 9   V4                  float64
 10  V5                  float64
 11  V6                  float64
 12  ritmi               float64
 13  age                 float64
 14  sex                 float64
 15  height              float64
 16  weight              float64
 17  nurse               float64
 18  site                float64
 19  device              float64
 20  heart_axis          float64
 21  validated_by        float64
 22  second_opinion      float64
 23  validated_by_human  float64
 24  pacemaker           floa

In [4]:
df.columns

Index(['I', 'II', 'III', 'aVF', 'aVR', 'aVL', 'V1', 'V2', 'V3', 'V4', 'V5',
       'V6', 'ritmi', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'heart_axis', 'validated_by', 'second_opinion',
       'validated_by_human', 'pacemaker', 'strat_fold'],
      dtype='object')

In [5]:
# train-test split
X = df[['I','II','III','aVF','aVR','aVL','V1','V2','V3','V4','V5','V6','ritmi','age','sex','height','weight']]
y = df['ritmi']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size = 0.25, random_state = 1234)

In [5]:
# Random Forest

# Random Forest

In [6]:
# Plug in appropriate max_depth and random_state parameters
rf = RandomForestClassifier()
# rf_param_grid = {'n_estimators': [20], 'criterion': ['entropy'], 'max_depth': [20]} #0.9267653536506913
rf_param_grid = {'n_estimators': [45], 'criterion': ['entropy'], 'max_depth': [45]} #0.9868391563552115
rf_cv= GridSearchCV(rf,rf_param_grid,cv=7,n_jobs=-1)
rf_cv.fit(X_train,y_train)

print("Best Score:" + str(rf_cv.best_score_))
print("Best Parameters: " + str(rf_cv.best_params_))

Best Score:1.0
Best Parameters: {'criterion': 'entropy', 'max_depth': 45, 'n_estimators': 45}


In [7]:
y_train_pred = rf_cv.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   1009610
         1.0       1.00      1.00      1.00    800602
         2.0       1.00      1.00      1.00   1429170

    accuracy                           1.00   3239382
   macro avg       1.00      1.00      1.00   3239382
weighted avg       1.00      1.00      1.00   3239382



In [8]:
y_test_pred = rf_cv.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    335792
         1.0       1.00      1.00      1.00    267846
         2.0       1.00      1.00      1.00    476156

    accuracy                           1.00   1079794
   macro avg       1.00      1.00      1.00   1079794
weighted avg       1.00      1.00      1.00   1079794



## Deep learning models

In [9]:
#CNN
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout,MaxPooling1D,LSTM
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical

2022-12-05 14:29:00.443765: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 14:29:00.733001: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-05 14:29:00.801859: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 14:29:00.801904: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [10]:
#Reshape train and test data to (n_samples, 187, 1), where each sample is of size (187, 1)
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

In [11]:
y_test_old = y_test.copy()

In [12]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [13]:
y_train.shape[1]

3

## 1-D Convolution

In [14]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-12-05 14:30:33.274666: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-05 14:30:33.274918: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-05 14:30:33.274935: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (TEO-Tabeed): /proc/driver/nvidia/version does not exist
2022-12-05 14:30:33.275544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# fit network
history_1d = model.fit(X_train, y_train, epochs=2)
# evaluate model

Epoch 1/2
101231/101231 [==============================] - 152s 2ms/step - loss: 0.0080 - accuracy: 0.9978
Epoch 2/2
101231/101231 [==============================] - 151s 1ms/step - loss: 0.0018 - accuracy: 0.9997


In [16]:
_, accuracy = model.evaluate(X_test, y_test)

33744/33744 [==============================] - 24s 711us/step - loss: 2.6839e-05 - accuracy: 1.0000


In [17]:
pred_result = model.predict(X_test)
final_pred = [np.argmax(i) for i in pred_result]
print(classification_report(y_test_old,final_pred))

33744/33744 [==============================] - 22s 649us/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    335792
         1.0       1.00      1.00      1.00    267846
         2.0       1.00      1.00      1.00    476156

    accuracy                           1.00   1079794
   macro avg       1.00      1.00      1.00   1079794
weighted avg       1.00      1.00      1.00   1079794



## LSTM models

In [18]:
lstm_model = Sequential()
lstm_model.add(LSTM(100, input_shape=(X_train.shape[1],1)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(100, activation="relu"))
lstm_model.add(Dense(y_train.shape[1], activation="softmax"))
lstm_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 3)                 303       
                                                                 
Total params: 51,203
Trainable params: 51,203
Non-trainable params: 0
_________________________________________________________________


In [19]:
# fit network
history_lstm = lstm_model.fit(X_train, y_train, epochs=1)
# evaluate model

101231/101231 [==============================] - 532s 5ms/step - loss: 0.0021 - accuracy: 0.9992


In [20]:
_, accuracy = lstm_model.evaluate(X_test, y_test)

33744/33744 [==============================] - 75s 2ms/step - loss: 1.1063e-05 - accuracy: 1.0000


In [21]:
lstm_pred_result = lstm_model.predict(X_test)

33744/33744 [==============================] - 72s 2ms/step


In [22]:
final_pred_lstm = [np.argmax(i) for i in lstm_pred_result]

In [23]:
print(classification_report(y_test_old,final_pred_lstm))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    335792
         1.0       1.00      1.00      1.00    267846
         2.0       1.00      1.00      1.00    476156

    accuracy                           1.00   1079794
   macro avg       1.00      1.00      1.00   1079794
weighted avg       1.00      1.00      1.00   1079794

